# Evaluation for the Real Pendulum Experiments
This notebook can be used to load and analyze the results generated by running `training_realPendulum.py`. The path to the the folder containing `ckpt.pth` and `./hydra/` needs to be specified under `path_experiment`.

In [ ]:
%cd ..

In [ ]:
import os
import yaml
import torch
from torchvision import utils
from models.sceneRepresentation import Scene
from dataset.dataset import ImageDataset_realData
from util.util import compute_psnr
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
# Set the path to the folder containing the results (/.hydra/, ckpt.pth)
path_experiment = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-24',
    'real_world',
    'pendulum',
    '10-20-22'
)

In [ ]:
# Load the config and the checkpoint
path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
with open(path_conf) as f:
    cfg = yaml.safe_load(f)

model = Scene(**cfg['scene']['background'])

model.add_pendulum(**cfg['ode'], **cfg['scene']['local_representation'])

path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
model.load_state_dict(torch.load(path_ckpt))

model.use_homography = cfg['homography']['enable']

model.to(device)
print("Moved to device")

In [ ]:
# Load the data
data_path = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])

train_data = ImageDataset_realData(
    **cfg['data'],
    normalize_by_H=True,
    max_samples=cfg['data']['samples_train']
)

eval_data = ImageDataset_realData(
    **cfg['data'],
    normalize_by_H=True,
    indices=train_data.unused_inds
)

tspan_eval = eval_data.t_steps.to(device)
print(f"Tspan eval data: {tspan_eval}")
print(f"Tspan train data: {train_data.t_steps}")

In [ ]:
# Render images and evaluate
model.eval()
model.update_trafo(tspan_eval)
H, W = eval_data.get_image_dim()
output = model.render_image(W, H, normalize_by_H=True)
ims = output["Image"].cpu()
masks = output['Mask'].cpu()

psnr = compute_psnr(ims, eval_data.get_full_images())
norm = torch.norm(torch.eye(3) - model.homography_matrix.cpu())

measured_l = 0.271
rel_error_l = torch.abs(model.local_representation.ode.l_pendulum - measured_l) / measured_l
print(f"PSNR: {psnr}, Norm diff: {norm}")
print(f"Rel Error l: {100*rel_error_l:.2f}%")
print(f"{psnr:.2f} & {norm:.2f}")

In [ ]:
# Save the renderings to the experiments folder
path_folder_rendering = os.path.join(path_experiment, 'renderingsRealWorld')
os.makedirs(path_folder_rendering)

width = int(ims.shape[2] / 2 - 2)

inds_to_save = [i for i in range(8, 15) if (i+1)%1==0]
print(f"Saving imgaes for timesteps {tspan_eval[inds_to_save]}")

# Save individual images
for i in inds_to_save:
    path = os.path.join(path_folder_rendering, f"{i}_eval.jpg")
    cur_im = ims[i].permute(2, 0, 1)
    utils.save_image(cur_im, path)

    path = os.path.join(path_folder_rendering, f"{i}_gt.jpg")
    cur_im_gt = eval_data.get_full_images(i).permute(2, 0, 1)
    utils.save_image(cur_im_gt, path)

    path = os.path.join(path_folder_rendering, f"{i}_mask_eval.jpg")
    cur_mask = masks[i]
    utils.save_image(cur_mask, path)

    path = os.path.join(path_folder_rendering, f"{i}_merged_gtRight.jpg")
    merged_im = torch.zeros_like(cur_im)
    merged_im[0, :, :] = 1.0
    merged_im[:, :, :width] = cur_im[:, :, :width]
    merged_im[:, :, -width:] = cur_im_gt[:, :, -width:]
    utils.save_image(merged_im, path)